# Ziling Zhang HW_2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests


%matplotlib inline

## Q1

In [2]:
def get_data(token, exchange):
    res = requests.get(
        f'https://api.cryptowat.ch/markets/{exchange}/{token}usd/ohlc',
        params={
            'periods': '3600',
            'after': str(int(pd.Timestamp('2021-11-22').timestamp()))
        }
    )

    df = pd.DataFrame(
        res.json()['result']['3600'],
        columns=['ts', 'open', 'high', 'low', 'close', 'volume', 'volumeUSD']
    )
    df['ts'] = pd.to_datetime(df.ts, unit='s')
    df['token'] = token
    
    return df

In [3]:
tokens = ['ETH', 'SOL', 'AVAX', 'USDT', 'FLOW']
exchanges = ['coinbase-pro', 'coinbase-pro', 'coinbase-pro', 'coinbase-pro', 'kraken']

In [4]:
dfs = [get_data(token, exchange) for token, exchange in zip(tokens, exchanges)]

In [5]:
df = pd.concat(dfs).set_index('ts')
df.head()

,open,high,low,close,volume,volumeUSD,token
ts,,,,,,,
2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH
2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH
2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH
2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH
2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH


In [6]:
len(df)

2825

## Q2

In [7]:
USD_volume = df.groupby('token')['volumeUSD'].sum().sort_values

In [8]:
USD_volume = df.groupby('token')['volumeUSD'].sum().sort_values(ascending = False)

In [9]:
USD_volume.to_frame()

,volumeUSD
token,
ETH,2.176907e+10
SOL,7.147179e+09
AVAX,3.569897e+09
USDT,2.031328e+09
FLOW,8.926501e+07


## Q3

In [10]:
ETH_SOL_close_ratio = df.groupby('ts', as_index=False).apply(lambda x: x[x['token'] == 'ETH'].close/x[x['token'] == 'SOL'].close)

In [11]:
ETH_SOL_close_ratio.to_frame().head()

,,close
,ts,
0,2021-11-22 00:00:00,18.471694
1,2021-11-22 01:00:00,18.736150
2,2021-11-22 02:00:00,18.577985
3,2021-11-22 03:00:00,18.794937
4,2021-11-22 04:00:00,18.891059


In [12]:
ETH_SOL_close_ratio.index = ETH_SOL_close_ratio.index.droplevel(0)

In [13]:
ETH_SOL_close_ratio = ETH_SOL_close_ratio.to_frame().head()
ETH_SOL_close_ratio.head()

,close
ts,
2021-11-22 00:00:00,18.471694
2021-11-22 01:00:00,18.736150
2021-11-22 02:00:00,18.577985
2021-11-22 03:00:00,18.794937
2021-11-22 04:00:00,18.891059


In [14]:
ETH_SOL_close_ratio.rename(columns={"close":"ratio_ETH_SOL"},inplace=True) 

In [15]:
df = df.join(ETH_SOL_close_ratio, on='ts')
df.head()

,open,high,low,close,volume,volumeUSD,token,ratio_ETH_SOL
ts,,,,,,,,
2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH,18.471694
2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH,18.736150
2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH,18.577985
2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH,18.794937
2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH,18.891059


## Q4

In [16]:
df.rename(columns={'volume':'volumeBase', 'volumeUSD': 'volumeTerm'},inplace=True)
df.head()

,open,high,low,close,volumeBase,volumeTerm,token,ratio_ETH_SOL
ts,,,,,,,,
2021-11-22 00:00:00,4317.98,4342.24,4246.07,4262.99,7262.562789,3.123077e+07,ETH,18.471694
2021-11-22 01:00:00,4263.04,4270.34,4212.45,4234.37,8437.946084,3.574586e+07,ETH,18.736150
2021-11-22 02:00:00,4234.87,4246.72,4171.17,4217.89,9259.725370,3.895639e+07,ETH,18.577985
2021-11-22 03:00:00,4217.88,4223.48,4163.58,4193.47,9259.899519,3.882032e+07,ETH,18.794937
2021-11-22 04:00:00,4192.95,4213.59,4147.00,4168.35,7934.546906,3.315476e+07,ETH,18.891059


## Q5

In [17]:
fat_table = pd.pivot_table(df, values='close', index='ts', columns=['token'],)

In [18]:
fat_table.head()

token,AVAX,ETH,FLOW,SOL,USDT
ts,,,,,
2021-11-22 00:00:00,128.67,4262.99,14.150,230.785,1.0012
2021-11-22 01:00:00,134.85,4234.37,13.621,226.000,1.0013
2021-11-22 02:00:00,132.54,4217.89,13.753,227.037,1.0012
2021-11-22 03:00:00,133.38,4193.47,13.652,223.117,1.0011
2021-11-22 04:00:00,136.29,4168.35,13.581,220.652,1.0009


## Q6

In [19]:
Return = fat_table.apply(lambda x: np.log(x)-np.log(x.shift(1)))
Return.head()

token,AVAX,ETH,FLOW,SOL,USDT
ts,,,,,
2021-11-22 00:00:00,NaN,NaN,NaN,NaN,NaN
2021-11-22 01:00:00,0.046912,-0.006736,-0.038102,-0.020952,0.0001
2021-11-22 02:00:00,-0.017279,-0.003900,0.009644,0.004578,-0.0001
2021-11-22 03:00:00,0.006318,-0.005806,-0.007371,-0.017417,-0.0001
2021-11-22 04:00:00,0.021583,-0.006008,-0.005214,-0.011109,-0.0002


## Q7

In [20]:
corr_mat_return = Return.corr()
corr_mat_return

token,AVAX,ETH,FLOW,SOL,USDT
token,,,,,
AVAX,1.000000,0.642650,0.390959,0.619992,-0.018064
ETH,0.642650,1.000000,0.565507,0.742477,-0.119233
FLOW,0.390959,0.565507,1.000000,0.449759,-0.192447
SOL,0.619992,0.742477,0.449759,1.000000,0.099878
USDT,-0.018064,-0.119233,-0.192447,0.099878,1.000000


## Q8

In [21]:
corr_mat_return.style.background_gradient(cmap='coolwarm')

token,AVAX,ETH,FLOW,SOL,USDT
token,,,,,
AVAX,1.000000,0.642650,0.390959,0.619992,-0.018064
ETH,0.642650,1.000000,0.565507,0.742477,-0.119233
FLOW,0.390959,0.565507,1.000000,0.449759,-0.192447
SOL,0.619992,0.742477,0.449759,1.000000,0.099878
USDT,-0.018064,-0.119233,-0.192447,0.099878,1.000000
